# Setup

In [1]:
#https://www.kaggle.com/piotrstanczyk/gfootball-template-bot  G-FOOTBALL TEMPLATE BOT
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.8 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

Cloning into 'kaggle-environments'...
remote: Enumerating objects: 4196, done.
remote: Counting objects: 100% (976/976), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 4196 (delta 779), reused 822 (delta 688), pack-reused 3220
Receiving objects: 100% (4196/4196), 11.38 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (2538/2538), done.
Processing /content/kaggle-environments
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 63 kB 140 kB/s 
  Created wheel for kaggle-environments: filename=kaggle_environments-1.9.6-py3-none-any.whl size=1811221 sha256=aed5514f3bdfca51086334e8a48510d2ee

# All the imports

In [2]:
from gfootball.env.football_env import FootballEnv
from kaggle_environments import make
from gfootball.env.config import Config
from gfootball.env.football_env import FootballEnv

#import dqn libraries
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import numpy as np
import pandas as pd
import itertools
import random
from collections import deque

#import env 
import gym
import gfootball 

#env_name = "GFootballBase-v0"
#print(env_name)

# Deep Q Network Model

In [3]:
class DQN(nn.Module):
  def __init__(self,env):
    super(DQN,self).__init__()
    input_dims = int(np.prod(env.observation_space.shape)) #neurons input layer = number of observations
    self.net =nn.Sequential(nn.Linear(input_dims,130), 
                            nn.Tanh(),
                            nn.Linear(130,env.action_space.n)) # neurons outpul layer = number of observations 
                            
  def forward(self,x):
    return self.net(x) #use the dqnetwork

  def act(self,obs): 
    obs_t =torch.as_tensor(obs,dtype=torch.float32)
    q_values=self(obs_t.unsqueeze(0)) # make tensor a batch dimension
    max_q_index = torch.argmax(q_values,dim=1)[0]   # taking action with highest q value
    action = max_q_index.detach().item # making tensor to integer which represents action 
    print (action)
    return action 



# Initialize variables

In [4]:
BUFFERSIZE = 50000
REWBUFFERSIZE = 100
MINREPLAYSIZE= 10 # Episode is 3000 steps
GAMMA = 0.04
EPSILON =0.03

EPSILON_START=1.0
EPSILON_END =0.2
EPSILON_DECAY=10000

BATCH_SIZE = 3


# Agent 

In [5]:
class Agent() : 
  def __init__(self,gamma,epsilon):
    self.gamma =gamma
    self.epsilon =epsilon
    self.ReplayBuffer = deque(maxlen=BUFFERSIZE) #Store experiences 
    self.RewBuffer = deque(maxlen=REWBUFFERSIZE) #Store rewards
    
    self.online_net = DQN(env)   
    self.target_net = DQN(env)  

    self.target_net.load_state_dict(self.online_net.state_dict())

  def transition(self,obs,new_obs,action,reward ,done): # should be tuple ? 
    self.obs =obs
    self.action=action
    self.reward =reward 
    self.done=done
    #self.info =info
    self.new_obs =new_obs
    TransitionTuple= (obs,new_obs,action,reward,done)
    #print("class",TransitionTuple)
    return TransitionTuple



# Create Environment -- Main 


In [8]:
#if __name__ =='main':


env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0") #List with the 115 states 
eps_reward =0.0
agent=Agent(GAMMA,EPSILON) #Create OBJECT from Agent class
obs = env.reset()

#Initialize ReplayBuffer
for i in range(MINREPLAYSIZE): 
  action =env.action_space.sample() #random action 

  new_obs ,reward,done ,info = env.step(action) 

  transition = agent.transition(obs,new_obs,action,reward ,done) #obs ,action ,reward , done ,info , new_obs  PROSEKSE TO MALLON LATHOS
  agent.ReplayBuffer.append(transition)  #Fill Replay Buffer with transitions
  obs=new_obs

  #eps_reward = +reward
  #print("|-------|",info ,"for i:",i,"|-------|")
  if(done):  # if someone score  a goal reset the env 
    obs=env.reset()

# MAIN TRAIN LOOP
obs =env.reset()
c= 0
for step in itertools.count():#WHILE TRUE LOOP 
  epsilon = np.interp(step,[0,EPSILON_DECAY],[EPSILON_START,EPSILON_END]) #Epsilon start->end with epsilon decays steps from 100% random actions->2% rnd actions
  rnd_sample = random.random()

  if rnd_sample <= epsilon: #random action |explore|
    action = env.action_space.sample()
  else:
    action = agent.online_net.act(obs) #best action |exploit|
    

  print(env.step(action))
  new_obs,reward,done,info = env.step(action)
  #print("MPIKE TOSES FORES ",c)
  #c=c+1
  transition = agent.transition(obs,new_obs,action,reward ,done) #fill Replaybuffer with transitions 
  agent.ReplayBuffer.append(transition)
  obs=new_obs

  eps_reward = +reward
  
  if (done) :
    obs=env.reset()

    agent.RewBuffer.append(eps_reward)
    eps_reward =0.0 


# Start Gradient Step 
  transitions =random.sample(agent.ReplayBuffer , BATCH_SIZE) #sample batch_size number of random transitions from Replaybuffer ,
                                                              # Replay buffer have been filled earlier
  #Store observations as arrays
  obses = np.asarray([t[0] for t in transitions])
  actions = np.asarray([t[1] for t in transitions])
  rewards = np.asarray([t[2] for t in transitions])
  dones = np.asarray([t[3] for t in transitions])
  new_obses = np.asarray([t[4]for t in transitions])

  #Convert observation arrays to pytorch tensors
  obses_t=torch.as_tensor(obses,dtype=torch.float32)
  actions_t = torch.as_tensor(actions,dtype=torch.int64)
  rewards_t = torch.as_tensor(rewards,dtype= torch.float32)
  dones_t = torch.as_tensor(dones,dtype= torch.float32)
  new_obses_t = torch.as_tensor(new_obses,dtype= torch.float32)

  #Compute Targets
  target_q_values = agent.target_net(new_obses_t)
  #max_target_q_values


print(transition)
  #if(i==3):
   # break'''

(array([-1.01102936e+00, -0.00000000e+00, -4.26654398e-01, -1.98944613e-01,
       -5.05514681e-01, -6.45939931e-02, -5.05514681e-01,  6.45929798e-02,
       -4.26654398e-01,  1.98944613e-01, -1.86243743e-01, -1.07399188e-01,
       -2.68472731e-01, -4.61845457e-06, -1.86243743e-01,  1.07399188e-01,
       -1.00441575e-02, -2.19616979e-01, -4.88684922e-02,  1.33812007e-11,
       -1.01102944e-02,  2.19617531e-01,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  3.74808023e-03, -1.23999380e-05,
        0.00000000e+00, -0.00000000e+00,  6.61370519e-04, -1.48298641e-05,
        3.61304940e-03,  2.65041183e-11,  0.00000000e+00, -0.00000000e+00,
        1.01102936e+00,  0.00000000e+00,  4.26654398e-01,  1.98944613e-01,
        5.04868567e-01,  6.45219460e-02,  5.05514681e-01, -6.45929798e-02,
        4.26007748e-01, 

AssertionError: ignored

In [ ]:
env = make("football", debug=True, configuration={"save_video": False, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})

Staring a new environment 1eea7a5c-7ecf-4e15-b672-84c6c4e89b55: with scenario: 11_vs_11_kaggle


In [ ]:
ena =env.reset() 
#dyo=simple_env.reset()

#print(ena.shape)
print(ena[0])#olo to environment
#pprint.pprint(ena[0])


Resetting environment cc712d27-0946-4f08-b0ae-bad76c935e40: with scenario: 11_vs_11_kaggle
{'action': [], 'reward': 0, 'info': {}, 'observation': {'remainingOverageTime': 60, 'step': 0, 'controlled_players': 1, 'players_raw': [{'left_team': [[-1.0110293626785278, -0.0], [-0.4266543984413147, -0.19894461333751678], [-0.5055146813392639, -0.06459399312734604], [-0.5055146813392639, 0.06459297984838486], [-0.4266543984413147, 0.19894461333751678], [-0.18624374270439148, -0.10739918798208237], [-0.270525187253952, -0.0], [-0.18624374270439148, 0.10739918798208237], [-0.010110294446349144, -0.21961550414562225], [-0.05055147036910057, -0.0], [-0.010110294446349144, 0.21961753070354462]], 'right_team_direction': [[-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0]], 'right_team_roles': [0, 2, 1, 1, 3, 5, 5, 5, 6, 9, 7], 'right_team_tired_factor': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *

## @human_readable_agent wrapper modifies raw observations 
# provided by the environment:
# https://github.com/google-research/football/blob/master/gfootball/doc/observation.md#raw-observations
# into a form easier to work with by humans.
# Following modifications are applied:
# - Action, PlayerRole and GameMode enums are introduced.
# - 'sticky_actions' are turned into a set of active actions (Action enum)
#    see usage example below.
# - 'game_mode' is turned into GameMode enum.
# - 'designated' field is removed, as it always equals to 'active'
#    when a single player is controlled on the team.
# - 'left_team_roles'/'right_team_roles' are turned into PlayerRole enums.
# - Action enum is to be returned by the agent function.
@human_readable_agent
def agent(obs):
    # Make sure player is running.
    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs['left_team'][obs['active']]
    # Does the player we control have the ball?
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:  #EXEI PAIKTIS TIN MPALA KAI EINAI STIN ARISTERI OMADA
        # Shot if we are 'close' to the goal (based on 'x' coordinate).
        if controlled_player_pos[0] > 0.5:
            return Action.Shot
        # Run towards the goal otherwise.
        return Action.Right
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_pos[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_pos[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_pos[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_pos[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide

Overwriting submission.py


In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py", "do_nothing"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

Left player: reward = None, status = ERROR, info = {}
Right player: reward = 100, status = DONE, info = {'debug_info': 'Opponent forfeited. You win.'}
